In [1]:
import mariadb
import sys

In [2]:
# Connect Mariadb
try:
    conn = mariadb.connect(
        user="root",
        password="123456",
#         host="127.0.0.1",
#         port=3306,
    )
except mariadb.Error as e:
    print(f"Error connecting to MariaDB Platform: {e}")
    sys.exit(1)

In [3]:
DB_NAME = 'wse'

TABLES = {}
TABLES['lexicon'] = (
    "CREATE TABLE `lexicon` ("
    "  `term`   varchar(30) NOT NULL,"
    "  `start`  bigint      NOT NULL,"
    "  `end`    bigint      NOT NULL,"
    "  `freq`   int         NOT NULL,"
    "  PRIMARY KEY (`term`)"
    ") ENGINE=InnoDB")

TABLES['docs'] = (
    "CREATE TABLE `docs` ("
    "  `url`   varchar(1024)   NOT NULL,"
    "  `text`  varchar(2048)   NOT NULL,"
    "  PRIMARY KEY (`url`)"
    ") ENGINE=InnoDB")

In [4]:
# Get Cursor
cur = conn.cursor()

In [5]:
# Create Database
def create_database(cursor):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(DB_NAME))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

try:
    cur.execute("USE {}".format(DB_NAME))
except conn.Error as err:
    print("Database {} does not exists.".format(DB_NAME))
    create_database(cur)
    print("Database {} created successfully.".format(DB_NAME))
    conn.database = DB_NAME

In [6]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cur.execute(table_description)
    except conn.Error as err:
        print(err.msg)

Creating table lexicon: Table 'lexicon' already exists
Creating table docs: Table 'docs' already exists


In [ ]:
# build lexicon
with open('/Volumes/Elements/indexer/lexicon') as lex:
    for line in lex:
        term, start, end, freq = line.strip().split()
        try:
            cur.execute(
                "INSERT INTO lexicon (term, start, end, freq) VALUES (?, ?, ?, ?)", 
                (term, int(start), int(end), int(freq)))
        except conn.Error as err:
            pass
conn.commit()

In [ ]:
# # check one term
# term = 'cheng'
# cur.execute(
#     "SELECT start, end, freq FROM lexicon WHERE term=?", 
#     (term,))

In [7]:
cur.execute("select COUNT(*) from lexicon")
for a in cur:
    print(a)

(36178733,)


In [ ]:
# build page table
with open('/Volumes/Elements/indexer/msmarco-docs.trec') as d:
    cnt = 0
    while True:
        nxt = d.readline()
        if len(nxt) == 0:
            break
        if nxt.strip() == "<DOC>":
            for i in range(2):
                d.readline()
            url = d.readline().strip()
            text = ''
            line = ''
            while True:
                line = d.readline().strip();
                if line == "</TEXT>":
                    break
                text += line + ' '
                try:
                    cur.execute(
                        "INSERT INTO docs (url, text) VALUES (?, ?)", 
                        (url, text[:2048])
                except conn.Error as err:
                    pass
conn.commit()

In [10]:
cur.execute("select COUNT(*) from docs")
for a in cur:
    print(a)

(3212965,)


In [ ]:
# # check one url
# url = 'http://internet-eraser-soft.tripod.com/address.html'
# cur.execute(
#     "SELECT text FROM docs WHERE url=?", 
#     (url,))

In [ ]:
conn.close()